In [139]:
from bs4 import BeautifulSoup, SoupStrainer
import requests
import urllib.request
import copy
import pandas as pd
from newspaper import Article
import re
import nltk
from pymorphy2 import MorphAnalyzer
from nltk.corpus import stopwords
import string
import pandas as pd
data = pd.read_csv('vocab.csv')
stopwords_ru=list(data['0'])
# символы для удаления
patterns = "[0-9!#$%&'()*+,./:;<=>?@[\]^_`{|}~—\"\-]+"
morph = MorphAnalyzer()

In [140]:
# лемматизация
def lemmatize(doc):
    doc = re.sub(patterns, ' ', doc)
    tokens = []
    print("len=",len(doc.split()))
    for token in doc.split():
        if token and token not in stopwords_ru:
            token = token.strip()
            token = morph.normal_forms(token)[0]
            tokens.append(token)
    print("//////////////////=",len(tokens))
    if len(tokens) > 2:
        return tokens
    return None

In [141]:
# security lab 
# выясняем сколько страниц на сайие
url="https://www.securitylab.ru/news/page1_1.php"
headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:100.0) Gecko/20100101 Firefox/100.0'}
result = requests.get(url, headers=headers)
# поиск по тегам
soup = BeautifulSoup(result.text, 'html.parser', parse_only=SoupStrainer('div'))
box = soup.find('div', class_='page-picker')
st = str(box.find('input'))
flag=0
num_page=''
for i in range(12,len(st)):
    if st[i]=='"':
        break
    else:
        num_page=num_page+st[i]
num_page=int(num_page)
num_page=1

In [142]:
# # выясняем сколько страниц на сайие
# url="https://www.securitylab.ru/news/page1_1.php"
# headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:100.0) Gecko/20100101 Firefox/100.0'}
# result = requests.get(url, headers=headers)
# # поиск по тегам
# soup = BeautifulSoup(result.text, 'html.parser', parse_only=SoupStrainer('div'))
# box = soup.find('div', class_='page-picker')
# st = str(box.find('input'))
# num_page=int(box)
# links = [link['href'] for link in box.find_all(href=True)]
# num_page=''
# for i in range (6, len(links[1])):
#     if links[1][i]=='&':
#         break
#     else:
#         num_page=num_page+links[1][i]
# num_page=int(num_page)

In [143]:
# получить контент
def get_news(url):
    news = Article(url, language='ru')
    news.download()
    news.parse()
    data_frame=pd.DataFrame()
    el=str(news.text)
    text = copy.deepcopy(el.lower())
    data_frame=pd.DataFrame()
    data_frame=pd.concat([data_frame,pd.DataFrame([text])])
    data = data_frame[0].apply(lemmatize)
#     el=str(news.text)
#     print(el[el.find("\n",el.find("\n")+2)+2:])

In [144]:
# собираем сслки на статьи на странице каждой
dataframe=pd.DataFrame()
for k in range(1,num_page+1):
    url="https://www.securitylab.ru/news/page1_"+str(k)+".php"
    headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:100.0) Gecko/20100101 Firefox/100.0'}
    result = requests.get(url, headers=headers)
    soup = BeautifulSoup(result.text, 'html.parser', parse_only=SoupStrainer('div'))
    box = soup.find_all('a', class_='article-card inline-card')
    link=[]
    for i in range (0, len(box)):
        try:
            stroka_url=str(box[i])
            urls=''
            for j in range (42,len(stroka_url)):
                if stroka_url[j]!='"':
                    urls=urls+stroka_url[j]
                else:
                    get_news("https://www.securitylab.ru"+urls)
#                     link.append(copy.deepcopy(urls))
                    break
        except ValueError:
            e=1
# print(link)

len= 238
//////////////////= 218
len= 186
//////////////////= 173
len= 149
//////////////////= 138
len= 123
//////////////////= 117
len= 164
//////////////////= 157
len= 185
//////////////////= 167
len= 184
//////////////////= 175
len= 315
//////////////////= 295
len= 262
//////////////////= 239
len= 102
//////////////////= 97
len= 251
//////////////////= 223
len= 126
//////////////////= 117
len= 221
//////////////////= 206
len= 353
//////////////////= 306
len= 239
//////////////////= 219


In [145]:
# # собираем сслки на статьи на странице каждой
# dataframe=pd.DataFrame()
# for k in range(1,num_page+1):
#     url="https://digital.gov.ru/ru/events/monitoring/?words=&type=&directions=&start_date=&end_date=&page="+str(k)
#     headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:100.0) Gecko/20100101 Firefox/100.0'}
#     result = requests.get(url, headers=headers)
#     soup = BeautifulSoup(result.text, 'html.parser', parse_only=SoupStrainer('div'))
#     box = soup.find_all('li', class_='')
#     link=[]
#     for i in range (0, len(box)):
#         try:
#             links = [link['href'] for link in box[i].find_all(href=True)]
#             if links[0][0:11]=='/ru/events/' and len(links[0])>=11 and int(links[0][11]):
#                 get_news('https://digital.gov.ru'+links[0])
# #                 link.append(copy.deepcopy(links[0]))
#         except ValueError:
#             e=1
# #     print(link)

In [146]:
# headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:100.0) Gecko/20100101 Firefox/100.0'}
# html_page = requests.get('https://digital.gov.ru/ru/events/41570/',headers=headers).content
# soup = BeautifulSoup(html_page, 'lxml')
# whitelist = ['h1', 'h2', 'h3','strong','b','u','em','i','ins','p','br','hr']
# out = ""

# for t in soup.find_all(text=True):
#     if t.parent.name in whitelist:
#         out += '{} '.format(t)

# escape = ['\r', '\n', '\t', '\xa0']
# for e in escape:
#     out = out.replace(e, '')
# out

In [147]:
# url="https://digital.gov.ru/ru/events/41568/"
# headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:100.0) Gecko/20100101 Firefox/100.0'}
# result = requests.get(url, headers=headers)
# soup = BeautifulSoup(result.text, 'html.parser', parse_only=SoupStrainer('div'))
# box = soup.find_all('div', class_='col-lg-8 col-md-8 text-content')
# link=[]
# for i in range (0, len(box)):
#     print(box[i].find_all('p'))
#     print()
#     links = [link['p'] for link in box[i].find_all('p')]

In [148]:
# data = pd.read_csv('vocab.csv')
# list(data['0'])

In [7]:
from selenium import webdriver
from selenium.webdriver.common.by import By
driver = webdriver.Firefox()

driver.get('https://rg.ru/search?text=%D0%B1%D0%B5%D0%B7%D0%BE%D0%BF%D0%B0%D1%81%D0%BD%D0%BE%D1%81%D1%82%D1%8C&sort=date')

button = driver.find_element(By.CLASS_NAME, 'LoadMoreBtn_wrapper__8OS4F')
button.click()

In [8]:
import requests
import pandas as pd

params = {
    'ort': 'cp',
    'rt': 'cart',
    'cartridgeId': 'root/content/contents[0]/Results[0]'
}

headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:100.0) Gecko/20100101 Firefox/100.0'}



def main(url):
    with requests.Session() as req:
        req.headers.update(headers)
        allin = []
        for num in range(0, 675, 24):
            params['No'] = num
            r = req.get(url, params=params)
            for item in r.json()['Results'][0]['records']:
                allin.append([item.get('name', 'N/A'), item['detailsUrl']])

        df = pd.DataFrame(allin, columns=["Title", "Url"])
        print(df)


main(
    'https://rg.ru/search?text=%D0%B1%D0%B5%D0%B7%D0%BE%D0%BF%D0%B0%D1%81%D0%BD%D0%BE%D1%81%D1%82%D1%8C')

JSONDecodeError: [Errno Expecting value] <!DOCTYPE html><html><head><meta name="viewport" content="width=device-width"/><meta charSet="utf-8"/><link rel="manifest" href="/manifest.json"/><script>
      if (!window.YaHeaderBiddingSettings) {
        const adfoxBiddersMap = {
          criteo: "851770",
          betweenDigital: "873981",
          adriver: "998474", // Soloway
          myTarget: "874698",
          // 'adfox_imho-mobile': "1082275" // IMHO
          'adfox_imho-mobile': "1085976", // IMHO
          'adfox_imho-banners': "1443716", // IMHO desktop
          'adfox_adsmart': "1195710", // AdSmart
          'adfox_WiseUP': "1288243",
          buzzoola: "1224359",
          rtbhouse: "1352133",
          otm: "1421443",
          hybrid: "1597346"
        };
        const adUnits = [];
        const userTimeout = 1000;

        window.YaHeaderBiddingSettings = {
          biddersMap: adfoxBiddersMap,
          adUnits: adUnits,
          timeout: userTimeout,
          ca